# Coursera | Andrew Ng (05-week3)—序列模型和注意力机制

【第 5 部分-序列模型-第一周】在吴恩达深度学习视频基础上，笔记总结，添加个人理解。- ZJ
    
>[Coursera 课程](https://www.coursera.org/specializations/deep-learning) |[deeplearning.ai](https://www.deeplearning.ai/) |[网易云课堂](https://mooc.study.163.com/smartSpec/detail/1001319001.htm)


[CSDN]()：
   


---

# 序列模型和注意力机制（Sequence to sequence models）

## <font color=#0099ff>3.1 基础模型 (Basic models)
    
Hello and welcome to this final week of this course,as well as to the final week of this sequence offive courses in Deep Learning Specialization.You're nearly at the finish line.In this week, you'll hear about sequence and sequence models which areuseful for everything from machine translation to speech recognition.The start of the basic models,and then later this week, you hear about pin search,detention model, and we'll wrap up the discussion ofmodels audio data like speech. Let's get started.Let's say you want to input a French sentence like "Jane visite I'Afrique en septembre."And, you want to translate it tothe English sentence "Jane is visiting Africa in September."As usual, let's use x1 through x, in this case,five to represent the words and the input sequence,and we use y1 through y6 to represent the words in the output sequence.So, how can you train a neural network to input the sequence x,and output to sequence y.Well, here's something you could do.And, the ideas I'm about to present are mainly from these two papers,due to Ilya Sutskever,Oriol Vinyals, and Quoc V. Le,and that one by Kyunghyun Cho,Bart van Merrienboer, Calgar Gulcehre,Dzmitry Bahdanau, Fethi Bougares, Holger Schwenk, and Yoshua Bengio.First, let's have a network which we're going to call the encoder network,be built as a RNN and this could be a jiayou analysia,feeding the input French words, one word at a time.And, after ingesting the input sequence,the RNN then outputs a vector that represents the input sentence.After that, you can build a decoder network,which you might draw here, which takes as inputthe encoding output by the encoding network shown in black on the left.And then, can be trained to output the translation,one word at a time.And eventually, it opens the CD endof sequence and the sentence token upon which the decoder stops.And, as usual, we could take the generator tokens and feed themto the next set of phase of state in the sequence,like we were doing before when synthesizing texts using the language model.One of the most remarkable recent results in Deep Learning is that this model works.Given enough pairs of French and English sentences,if you train a model to inputa French sentence and output the corresponding English translation,this will actually work decently well.And, this model simply uses an encoder network whose job it is to find an encoding ofthe input French sentence and then usea decoder network to then generate the corresponding English translation.An architecture very similar to this also works for image captioning.So, given an image like the one shown here,maybe you want it to be captioned automatically as a cat sitting on a chair.So, how do you train a neural network to input an image andoutput a caption like that face up there.Here's what you can do. From the earlier course on conference,you see how you can input an image into a convolutional network,maybe a pretrained AlexNet.And have that learn an encoding,or learn a set of feature of the input image.So, this is actually the AlexNet architecture.And, if we get rid of this final Softmax unit,the pretrained AlexNet can give youa 4096 dimensional feature vector of which to represent this picture of a cat.And so, this pretrained network can be the encoder network for the imageand you now have a 4000 or 4096 initial vector that represents the image.You can then take this and feed it to an RNN,whose job with it is to generate the caption one word at a time.So, similar to what we saw with machine translation,translating from French to English,you can now input a feature vector describing the inputand then have it generate an output sequence,output set of words one word at a time.And that actually works pretty well for image captioning,especially if the caption you want to generate is not too long.As far as I know, this type of model was first proposed by Junhua Mao,Wei Xu, Yi Yang, Jiang Wang,Zhiheng Huang, and Alan Yuille.Although it turns out there are multiple groups come out with very similar modelsindependently and at about the same time.So, two other groups that had donevery similar work at about the same time and I think independently are Oriol Vinyals,Alexander Toshev, Samy Bengio,and Dumitru Erhan as well as Andrej Karpathy and Fei-Fei Li.So, you've now seen how a basic sequence,a sequence model work,so how basic image the sequence or image captioning model works.But there are some differences between how yourun a model like this to generate the sequencecompared to how you were synthesizing novel text using a language model.One of the key differences is,you don't want a randomly chosen translation.You maybe want the most likely translation.You don't want the randomly chosen caption. Maybe not.But you might want the best caption and most likely caption.So let's see in the next video how you go about generating that.    
    
    
---
## <font color=#0099ff> 3.2 选择最有可能的句子(Picking the most likely sentence)
    
There are some similarities between the sequence to sequence machine translation modeland the language models that you have worked within the first week of this course,but there are some significant differences as well.Let's take a look. So, you can think ofmachine translation as building a conditional language model.Here's what I mean, in language modeling,this was the network we had built in the first week.And this model allows you to estimate the probability of a sentence.That's what a language model does.And you can also use this to generate novel sentences,and sometimes when you are writing x1 and x2 here,where in this example,x2 would be equal to y1 or equal to y and one is just a feedback.But x1, x2, and so on were not important.So just to clean this up for this slide,I'm going to just cross these off.X1 could be the vector of all zeros and x2,x3 are just the previous output you are generating.So that was the language model.The machine translation model looks as follows,and I am going to use a couple different colors,green and purple, to denote respectivelythe coded network in green and the decoded network in purple.And you notice that the decoded network looks pretty muchidentical to the language model that we had up there.So what the machine translation model is,is very similar to the language model,except that instead of always starting along with the vector of all zeros,it instead has an encoded networkthat figures out some representation for the input sentence,and it takes that input sentence and starts off the decoded network withrepresentation of the input sentence rather than with the representation of all zeros.So, that's why I call this a conditional language model,and instead of modeling the probability of any sentence,it is now modeling the probability of, say,the output English translation,conditions on some input French sentence.So in other words, you're trying to estimate the probability of an English translation.Like, what's the chance that the translation is "Jane is visiting Africa in September,"but conditions on the input French censors like,"Jane visite I'Afrique en septembre."So, this is really the probability of an English sentence conditions onan input French sentence which is why it is a conditional language model.Now, if you want to apply this model to actuallytranslate a sentence from French into English,given this input French sentence,the model might tell you what is the probabilityof difference in corresponding English translations.So, x is the French sentence,"Jane visite l'Afrique en septembre."And, this now tells you what is the probability ofdifferent English translations of that French input.And, what you do not want is to sample outputs at random.If you sample words from this distribution,p of y given x, maybe one time you get a pretty good translation,"Jane is visiting Africa in September."But, maybe another time you get a different translation,"Jane is going to be visiting Africa in September. "Which sounds a little awkward but is not a terrible translation,just not the best one.And sometimes, just by chance,you get, say, others: "In September,Jane will visit Africa."And maybe, just by chance,sometimes you sample a really bad translation:"Her African friend welcomed Jane in September."So, when you're using this model for machine translation,you're not trying to sample at random from this distribution.Instead, what you would like is to find the English sentence,y, that maximizes that conditional probability.So in developing a machine translation system,one of the things you need to do is come up with an algorithm that can actually findthe value of y that maximizes this term over here.The most common algorithm for doing this is called beam search,and it's something you'll see in the next video.But, before moving on to describe beam search,you might wonder, why not just use greedy search? So, what is greedy search?Well, greedy search is an algorithm from computer science which says to generatethe first word just pick whatever isthe most likely first word according to your conditional language model.Going to your machine translation model and then after having picked the first word,you then pick whatever is the second word that seems most likely,then pick the third word that seems most likely.This algorithm is called greedy search.And, what you would really like is to pick the entire sequence of words, y1, y2,up to yTy, that's there,that maximizes the joint probability of that whole thing.And it turns out that the greedy approach,where you just pick the best first word,and then, after having picked the best first word,try to pick the best second word,and then, after that,try to pick the best third word,that approach doesn't really work.To demonstrate that, let's consider the following two translations.The first one is a better translation,so hopefully, in our machine translation model,it will say that p of y given x is higher for the first sentence.It's just a better, more succinct translation of the French input.The second one is not a bad translation,it's just more verbose,it has more unnecessary words.But, if the algorithm has picked "Jane is" as the first two words,because "going" is a more common English word,probably the chance of "Jane is going," given the French input,this might actually be higher than the chance of "Jane isvisiting," given the French sentence.So, it's quite possible that if you just pickthe third word based on whatever maximizes the probability of just the first three words,you end up choosing option number two.But, this ultimately ends up resulting in a less optimal sentence,in a less good sentence as measured by this model for p of y givenx. I know this was may be a slightly hand-wavey argument,but, this is an example of a broader phenomenon,where if you want to find the sequence of words, y1, y2,all the way up to the final word that together maximize the probability,it's not always optimal to just pick one word at a time.And, of course, the total number of combinations ofwords in the English sentence is exponentially larger.So, if you have just 10,000 words in a dictionary and if you'recontemplating translations that are up to ten words long,then there are 10000 to the tenth possible sentences that are ten words long.Picking words from the vocabulary size,the dictionary size of 10000 words.So, this is just a huge space of possible sentences,and it's impossible to rate them all,which is why the most common thing to do is use an approximate search out of them.And, what an approximate search algorithm does,is it will try,it won't always succeed,but it will to pick the sentence, y,that maximizes that conditional probability.And, even though it's not guaranteed to find the value of y that maximizes this,it usually does a good enough job.So, to summarize, in this video,you saw how machine translation can be posed as a conditional language modeling problem.But one major difference between this andthe earlier language modeling problems is ratherthan wanting to generate a sentence at random,you may want to try to find the most likely English sentence,most likely English translation.But the set of all English sentences of a certain lengthis too large to exhaustively enumerate.So, we have to resort to a search algorithm.So, with that, let's go onto the next video whereyou'll learn about beam search algorithm.    
    
    
    
---
## <font color=#0099ff> 3.3 定向搜索 (Beam Search)  
    
In this video you learn about the beam search algorithm.In the last video you remember how for machine translation, given an inputFrench sentence, you don't want to output a random English translation.You want to output the best,the most likely English translation.The same is also true for speech recognition, where given an inputaudio clip, you don't want to output a random text transcript of that audio.You want to output the best,maybe the most likely text transcript.Beam search is the most widely used algorithm to do this.And in this video, you see how to get beam search to work for yourself.Let's illustrate beam search using our running example of the French sentence[FOREIGN] hopefully, being translated into Jane visits Africa in September.Sorry, let's start this slide over again.Let's illustrate beam search using our running example of Jane visits Africa inSeptember.Let's describe beam search using our running example of the French sentence[FOREIGN] hopefully, being translated into Jane visits Africa in September.The first thing beam search is to do is try to pickthe first words of the English translation that's going to output.So here I've listed all say,10,000 words in the vocabulary.And to simplify the problem a bit, I'm going to ignore capitalizations.So I'm just listing all the words in lowercase.So in the first step of beam search,it'll use this network fragment where the encode is shown in green and the decode isshown in purple to try to evaluate what is the probability of that first word.So what's the probability of the first output y given the inputsentence x given the French input.And whereas, greedy search would pick the single most likely words andmove on, beam search, instead, tries several different alternatives.In particular, it can try three different alternatives.Here, three is a parameter of the algorithm.It's called the beam width parameter.And by setting B to a different values,you can get beam search to simultaneously consider.So whereas, greedy search will pick only the one most likely words andmove on, beam search instead can consider multiple alternatives.So the beam search algorithm has a parameter called B,which is called the beam width.And for this example, I'm going to set the beam width to be equal to 3.And what this means is beam search will consider not just one possibility butconsider three at a time.So in particular,let's say evaluating this probability over different choices the first words.It finds that the choices in jane and september are the most likelythree possibilities for the first word in the English output.Then beam search will store away in computermemory that it wants to try out all three of these words.And if the beam width was five, it would try five words.If the beam width was ten, it would try andkeep track of the ten most likely choices for the first words.Let me say that long sentence again.And if the beam width parameter was set differently, if the beam width parameterwas say ten, then it would keep track of not just three, butup to ten most likely possible choices for the first word.So to be clear,in order to perform this first stepof beam search, what you need to do is run the inputFrench sentence through this encoder network.And then this first step of the decoder network,if this is a softmax output overall 10,000 possibilities,then you would take those 10,000 possible outputs andkeep in memory which were the top three.Let's go into the second step of beam search.Having picked in jane and september as the the three most likelychoices of the first word,what beam search will do now is foreach of these three choices,consider what should be the second word.So after in, maybe the second word is a,or maybe it's aaron.I'm just listing words from the vocab here from the dictionary.Or somewhere down the list will be September.Somewhere down the list is visit.And then all the way to z, maybe the last word is zulu.And there could be an EOS character in there as well.Actually, Danielle, leave off the EOS comment please, yeah.So never mind that, okay.So to evaluate the probability of the second word,it will use this neural network fragment where there's the encoder in green.And for the decoder portion, when trying to decide what comes after in,remember the decoder first outputs y hat 1.So I'm going to set this y hat 1 to the word in and feed this back in.So there's the words in, because they'retrying to figure out that the first word was in, what is the second word?And then this will output, I guess y hat 2.And so by hard wiring y hat 1 here,really the input here, to be the first word in.This network fragment can be used to evaluate what is the probabilityof the second word given the input French sentence andthat the first word of the translation has been the word it.Now, notice that what we ultimately care about in this second step of beam searchis to find the pair of the first and second words that is most likely.So not just the second word that's most likely, but the pair of the first andsecond words that are most likely.And by the rules of conditional probability,this can be expressed as P of the first word timesP of the probability of the second word.Right, which you are getting from this neural network fragment.And so if for each of the three words you've chosen, in, jane, and september,you save away this probability.Then you can multiple them by this second probability to getthe probability of the first and second words.So now,you've seen how if the first word was in,how you can evaluate the probability of the second word.Now, if the first word was jane,you'd do the same thing.The sentence could be jane a,jane aaron, and soon down to jane is,jane visits, and so on.And you will use this neural network fragment.And let me draw this in as well where here you would hard wire y hat 1 to be jane.Shoot, I just realized I've forgotten to draw this arrow earlier.Maybe and use some video editing magic to make this horizontal arrow appear earlierin the video.All right, and so with the first word, y<1> hat,hardwired as jane,then this neural network fragment cantell you what's the probability the second word,given the input x and given that the first word was jane.And then same as above, you can multiplyit with p of y<1> to get the probability of y<1> and y<2> foreach of these 10,000 different possible choices for the second word.And then finally, you do the same thing for september.All the words from a down to zulu and use this network fragment.I'll just draw this in as well,to see if the first word was september,what are the most likely options for the second words.So for this second step of beam search,because we're continuing to use a beam width of three.And because there are 10,000 words in the vocabulary,you'd end up considering 3 times 10,000 or 30,000 possibilities,because there are 10,000 here,10,000 here, 10,000 here.So it's the beam width times the number of words in the vocabulary.[COUGH] And what you do is you evaluate all of these 30,000 options accordingto the probability of the first and second words.And then pick the top three.So really cut down these 30,000 possibilities down to three again,down to the beam width parameter again.And so let's say that these 30,000 choices, the most likely werein september, and say jane is,and jane visits.Sorry, this a bit messy, but those are the most likely three out ofthe 30,000 choices, then that's what beam search would memorize away andtake on to the next step of beam search.So notice one thing.If beam search decides that the three most likely choices for the first andsecond words are in september or jane is or jane visits,then what that means is that it is now rejecting september as a candidate forthe first word of the output English translation.So we're now down to two possibilities for the first word.But we still have a beam width of three keeping trackof three choices for pairs of y<1>, y<2>.So before going on to the third step of beam search,I just want to notice that because the beam width is equal to three,at every step you instantiate three copies of the network toevaluate these partial sentence fragments in the output.And it's because the beam width is equal to three that you have three copies ofthe network with different choices for the first word.But these three copies of the network can be very efficiently used to evaluateall 30,000 options for the second word.So just don't instantiate 30,000 copies of the network.You need only three copies of the network to very quickly evaluate all10,000 possible outputs at that softmax output, say, for y<2>.Let's just quickly illustrate one more step of beam search.Let's just quickly illustrate one more step of beam search.So we said that the most likely choices of first twowords were in september,jane is and jane visits.And for each of these pairs of words we should have savedaway in computer memory the probability of y<1> andy<2> given the input x,given the French sentence x.So similar to before we now want to consider what is the third word.So is it in september a,in september aaron,all the way down to in september zulu.And to evaluate possible choices for the third word,you use this network fragment where youhardwire the first here to be in the second word to be september.And so this network fragment allows you to evaluate what's the probability ofthe third word given the input French sentence x and given that the first twowords are in september in the English output.And then you do the same thing for the second fragment.So like so.And same thing for jane visits.And so beam search will then once again, pick the top three possibilities.Maybe it thinks in september jane is a likely outcome orjane is visiting is likely, or maybe jane visits africa is likely forthe first three words and then it keeps going.And then you go on to the fourth step of beam search to add one more word andon it goes.Let me just do that last piece again.And the outcome of this process hopefully will be that adding one word at a time.That beam search will decide that jane visits africa in september will beterminated by the end of sentence symbol if you're using that in your system,which is quite common.That it'll find that this is a likely output English sentence.And you see more details of this for yourself in this week's exercise as well,where you get to play with beam search yourself.So with a beam width of three, beam search considers three possibilities at a time.Notice that if the beam width was set to be equal to 1, so because there's only 1,then this essentially becomes the greedy search algorithm,which we had discussed in the last video.Then this essentially becomes the greedy search algorithmwhich we had discussed in the last video.But by considering multiple possibilities, say three or ten orsome other number at the same time,beam search will usually find a much better output sentence than greedy search.You've now seen how beam search works, but it turns out there's some additional tipsand tricks and refinements that help you to make beam search work even better.Let's go into the next video to take a look.    
    
    
    
---
## <font color=#0099ff> 3.4 改进定向搜索 (Refinements to Beam Search) 
    
WEBVTTIn the last video,you saw the basic beam search algorithm.In this video, you learned some little changes that'll make it work even better.Length normalization is a small change to the beam search algorithmthat can help you get much better results.Here's what it is.We talked about beam search as maximizing this probability.And this product here is just expressing the observation that P ofy1 up to y Ty given x canbe expressed as P of y1 given x,times P ofy2 given x and y1 times dot dot dot,up to I guess P of y, Ty given x, andy1 up to y Ty minus one.But then maybe this notation is a bit more scary, andmore intimidating than it needs to be.But this is at product of probabilities that you see previously.Now, rather than,now if you're implementing these,these probabilities are all numbers less than 1.In fact, often they are much less than 1.And multiplying a lot of numbers less than 1 will result in a tiny, tiny,tiny number,which can result in numerical underflow.Meaning that is too small forthe probability part representation in your computer to store accurately.So in practice, instead of maximizing this product, we will take logs.And if you insert a log there, then a log of a product becomes a sum of a log andmaximizing this sum of log probabilities should give you the sameresult in terms of selecting the most likely sentence y.So by taking logs,you end up with a more numerically stablealgorithm that is less prone to rounding errors,numerical rounding errors, or to really numerical underflow.And because the log function,that's the log grouping function,there's a strictly monotonically increasing function, maximizing P(y).And because the logarithmic function,here's a long function,is a strictly monotonically increasing function, we know that maximizinglog P(y) given x should give you the same result as maximizing P(y) given x,as in the same value of y that maximizes this should also maximize that.So in most implementations, you keep track of the,sum of logs of probabilities, rather than the product of the probabilities.Now, there is one other tweak to this.All right, there's another typo here.I hope you can do video editing magic.So we can look good from the start.Now there's one other change to this objective functionthat makes the machine translation algorithm work even better.Which is that if you refer to this original objective up here,if you have a very long sentence, the probability of that sentence is going tobe low because you're multiplying as many terms here plus a numbers andless than one to estimate the probability of that sentence.And so if you multiply a lot of numbers that are less than one together,you just tend to end up with a smaller probability.And so this objective function has an undesirable effect,that it maybe unnaturally tends to prefer very short translations,tends to prefer very short outputs.Because the probability of a short sentence is determined just bymultiplying fewer of these numbers less than one.And so the product will just be not quite as small.And by the way, the same thing is true for this.The log of a probability is always less than or equal to 1.You're actually in this major the lock, sothe more terms you had together, the more negative this thing becomes.So there's one other change the algorithm that makes it work better,which is instead of using this as the objective you're trying to maximize.One thing you can do is normalize this by the number of words in your translation.And so this takes the average of the log of the probability of each word andthis significantly reduces the penalty for outputting longer translations.And in practice, as a heuristic,instead of dividing by Ty, by the number of words in the output sentence.Sometimes you use the softer approach,where you have Ty to the power of alpha,where maybe alpha is equal to 0.7.So if alpha was equal to 1, then you're completely normalizing by length.If alpha was equal to 0 then well,Ty to the 0 would be 1,then you're just not normalizing at all.And this is somewhere in between full normalization and no normalization.And alpha is another parameter,alpha parameter of algorithm that you can tune to try to get the best of results.And half of it using alpha this way,this is a heuristic or this is a hack,there isn't a great theoretical justification for it butpeople have found this works well.People have found it works well in practice.So many groups will do this.And you can try out different values of alpha until, andsee which one gives you the best result.So just to wrap up how you run beam search.As you run beam search, you see a lot of sentences with length equal 1,a lot of sentences with length equal 2,a lot of sentences with lengthequals 3 and so on and maybe,you run beam search for 30 steps.You consider output sentences up to length 30, let's say.And so with beam width of 3,you'll be keeping track of the top threepossibilities for each of these possible sentence lengths.1, 2, 3, 4, and so on up to 30.Then you will look at all the, Outputsentences and score them against this score.If you're using an EOS,you could also artificially tag onthe end of sentence token to your best three choices.And so,you can take your top sentences andjust compute this objective function on the sentences that you have seenthrough the beam search process.And then finally, of all three sentences that you have added this way,you will pick the one that achieves the highest valueon this normalized log probability objective.Sometimes it's called a normalized likelihood objective.And then that will be the final translation you output.So that's how you implement beam search andyou get to play at this yourself in this week's program exercise.Finally, a few implementational details.How do you choose the beam width B?The larger B is, the more possibilities you're considering, andthus the better the sentence you probably find.But the larger B is, the more computationally expensive your algorithmis because you're also keeping a lot more possibilities around, right?So finally,let's just wrap up with some thoughts on how to choose the beam width B.So here are the pros and cons of setting B to be very large versus very small.If the B width is very large,then you consider a lot of possibilities.And so you tend to get a better resultbecause you're considering a lot of different options, but it will be slower.And the minimum requirements will also grow,it'll also be computationally slower.Whereas if you use a very small beam width, then you get a worse result.Because you're just keeping less possibilities in mindas the algorithm is running,but you get a result faster.And the memory requirements will also be lower.So in the previous video we use,in our running example,a beam width of three so we're keeping three possibilities in mind.In practice, that is on the small side.In production systems, it's not uncommon to see a beam width maybe around ten.And I think a beam width of 100 would be considered very large fora production system,depending on the application.But for research systems, where people want to squeeze out every last drop ofperformance in order to publish a paper the best possible result.It's not uncommon to see people use beam widths of 1,000 or 3,000.But this is very application,as well as domain dependent.So I would say, try out a variety of values of B and see what works foryour application.But when B gets very large,there is often diminishing returns.So for many applications, occasions,I will expect to see a huge gain as you gofrom a beam width of 1, which is basically a beam search, to 3, to maybe 10.But the gains as you go from 1,000 to 3,000 in beam width are might notbe as big and forthose of you that have taken maybe a lot of computer science courses before.If you're familiar with computer science search algorithms like BFS,Breadth-first search, or DFS, Depth-first search.The way to think about beam search is that unlike those other algorithms,which you might have learned about in a computer science algorithms course anddon't worry about it if you've not heard of these algorithms.But if you've heard of Breadth-first search and Depth-first search,then unlike those algorithms which are exact search algorithms,beam search runs much faster but is not guaranteed to find the exact maximum forthis arg max that you'd like to find.If you haven't heard of Breadth-first search or Depth-first search,don't worry about it, it's not important for our purposes.But if you have, this is how beam search release those algorithms.So that's it for beam search which is a widely used algorithm inmany production system or in many commercial systems.Now in the third course in the sequence of course of deep learning,we talk a lot about error analysis.It turns out, one of the most useful tools I found is to be able to doerror analysis on beam search.So you sometimes wonder,should I increase my beam width?Is my beam width working well enough?And there's some simple things we can compute to give youguidance on whether you need to work on improving your search algorithm.Let's talk about that in the next video.    
    
---
## <font color=#0099ff> 3.5 定向搜索的误差分析 (Error analysis in beam search) 


    
---
## <font color=#0099ff> 3.6 Bleu 得分 (Bleu Score) 
    
    
    
---
## <font color=#0099ff> 3.7 注意力模型直观理解 (Attention Model Intuition) 



    
    
---
## <font color=#0099ff> 3.8 注意力模型 (Attention Model)  
    
---
## <font color=#0099ff> 3.9 语音辨识 (Speech recognition)
    
---
## <font color=#0099ff> 3.10 触发字检测 (Trigger word detection)
    
    
    
---
## <font color=#0099ff> 3.11 结论和致谢 (Summary and thank you)  
    
    
  
    
    
---
 
**PS: 欢迎扫码关注公众号：「SelfImprovementLab」！专注「深度学习」，「机器学习」，「人工智能」。以及 「早起」，「阅读」，「运动」，「英语 」「其他」不定期建群 打卡互助活动。**

<center><img src="http://upload-images.jianshu.io/upload_images/1157146-cab5ba89dfeeec4b.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240"></center>